# Twitter Trends Monitor v.1.0
A simple Python app which connects to the Twitter API and fetches the top 50 trending topics for a list of locations. Optionally, it will send the trends to an Azure Event Hub.

## Install and import libraries

In [ ]:
# install libraries
!pip install geocoder
!pip install tweepy
!pip install azure-eventhub
!pip install opencensus-ext-azure

In [ ]:
# import libraries
import tweepy
import os
import json
import sys
import geocoder
import time
import codecs
from azure.eventhub import EventHubProducerClient, EventData
from azure.eventhub.exceptions import EventHubError
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler

## Configure connection strings to Twitter and Azure Event Hub

In [ ]:
# Twitter
consumer_key = "xxxxxxxxxxxxxxxxxxxxxxx"
consumer_secret = "xxxxxxxxxxxxxxxxxxxxxxx"
access_key = "xxxxxxxxxxxxxxxxxxxxxxx"
access_secret = "xxxxxxxxxxxxxxxxxxxxxxx"

# Azure Event Hub connection string and event hub name
connection_string = "xxxxxxxxxxxxxxxxxxxxxxx"
event_hub_name = "xxxxxxxxxxxxxxxxxxxxxxx"

## Set variables
Set the list of locations to look up trends, how frequently the twitter API endpoint will be accessed and for how long will the application execute.

In [ ]:
# Set the list of locations (i.e. cities, countries)
locations = ["Greece", "New York", "Paris"]

# How often will the application fetch the trends
queryIntervalSeconds = 30

# How long will the application execute
runtime = 2*60

# initialize tweepy client
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

## Main Code

In [ ]:
def send_event_data_batch(message):
    # sends captured trend JSON files as events to the Azure Event Hub
    producer = EventHubProducerClient.from_connection_string(conn_str=connection_string,eventhub_name=event_hub_name)
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(message))
    producer.send_batch(event_data_batch)
    producer.close()

In [ ]:
def saveTrendsToJson(output):
    # saves a trends response for a location to a json file
    print("saving output to json")
    with open('trends-output.json', 'a', encoding='utf8') as outfile:
        json.dump(output, outfile, ensure_ascii=False)
        #outfile.write(test)
        outfile.write("\n")
    return(None)

In [ ]:
def getTrendingTopics(locations):
    # Retrieves trending topics for each location and outputs a JSON file
    for i in locations:
        try:
            g = geocoder.osm(i)
            print("Fetcing trending topics for location:", g)
            closest_loc = api.trends_closest(g.lat, g.lng)
            trendingTopics = api.trends_place(closest_loc[0]['woeid'])
            trendingTopicsJson = json.dumps(trendingTopics)
            saveTrendsToJson(trendingTopics)
            # comment out below line to skip sending the topics list to the event hub
            send_event_data_batch(trendingTopicsJson)
            print("Successfully processed trending topics for location:", g)
        except KeyError:
            print("reached keyerror")
            continue
        except Exception:
            print("error:", sys.exc_info()[0])
            logger.warning("Unable to find location %s", g)
            continue

## Start fetching trends!
Execute this cell to initiate the Trends Monitor.

In [ ]:
start_time = time.time()
logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string='InstrumentationKey=d25e2379-7318-47c4-a9ad-178c869e13d8'))
print("Started tracking trending topics for",runtime,"seconds at UTC time",time.strftime("%H:%M:%S", time.localtime()), "and for",len(locations),"locations:",locations)
while True:
    if (time.time() - start_time) < runtime:
        trendingTopics = getTrendingTopics(locations)
        time.sleep(queryIntervalSeconds - ((time.time() - start_time) % queryIntervalSeconds))
    else:
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        print('Runtime limit of', runtime, ' seconds reached, stopping connection at UTC time.',current_time)
        sys.exit()